<a href="https://colab.research.google.com/github/OmerRosen/Kaggle/blob/main/IMDB_2022_Hit_or_Flop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 2022 - Hit or Flop

## Data Collection

### Data Requirments

# Feature collection mission

# Main list Page:
* Movie Id
* Movie Title
* Movie year
* Rating
* MetaScore - Outcome
* Description
* Poster
* Directore name + Link
* Stars
* Votes - Output
* Genre

# Movie Page:
* Movie Length
* Rank
* Writer
* Star #1
* Star #2
* Star #3
* User eviews
* Critic reviews
* Number of photos posted
* Storyline - Text
* Tag line
* Release Date - Month
* Country of origin
* Additional Lanaguages
* Country of filming
* Production companies
* Budget
* Opening Weekend Date
* Gross US & Canada - Output
* Opening weekend US & Canada - Output
* Gross worldwide - Output
* Color - Color
* Color - Black&White
* Sound mix - Dolby Digital
* Sound mix - Dolby Atmos
* Aspect ratio

# Director Page:
* Director Age
* Is Top 500?
* Director Gender (Based on bio)
* Previous film count - As Director
* Previous film count - As Writer
* Previous film count - As As Producer
* Director Publicity listing count

# Information Abount Cast:
* Full list of cast and their profile links
* Number of cast members
* Produced by - Is top 500?
* Music by - Is top 500?
* Cinematography by  - Is top 500
* Film Editing by   - Is top 500
* Art Direction by  - Is top 500
* Number of Production Management
* Number of Art Department
* Number of Sound Department
* Number of Camera and Electrical Department
* Number of Editorial Department
* Number of Music Department
* Number of Additional Crew

# Actor Page
* Is Top 5000?
* Is Top 500?
* Is Top 100 (aka - Has numberical rank)
* Is Top 10 (aka - Has numberical rank)
* Numerical Rank (Could be none)
* Gender
* Age
* Oscar nominations 
* Birth country

# Production COmpanies (Filter)
* Num of previous titles


# Aggragated data:
* How many male stars
* How many female stars
* Avrage cast memeber age
* Max/Min age of cast member
* Number of unique birth countries of actors
* Total number of Oscar numinatior for cast
* Num cast members in top 5000
* Num cast memebers in top 500
* Num cast members in top 100
* Num cast members in top 10

# IMDB Scraper

## Build functions for collecting movie details

In [1]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time

In [2]:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie
start_point = 1 # Start from movie #1-50

movie_search_url = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&{start_point}anguages=en&start=1&ref_=adv_nxt"
movie_search_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&1anguages=en&start=1&ref_=adv_nxt'

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

list_of_50_movies = requests.get(f'{movie_search_url}', headers=headers, timeout=10)
list_of_50_movies

<Response [200]>

In [4]:
list_of_50_movies_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser').find_all('div',{'class':'lister-item mode-advanced'})


In [5]:
movie_search_page = list_of_50_movies_soup[0]

In [6]:
def get_basic_details(movie_search_page):
  search_box_info = {}

  search_box_info['movie_place'] = None if not movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}) else movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}).text.replace('.','')
  search_box_info['movie_name'] = movie_search_page.find('a')
  search_box_info['movie_page_url'] = f"{base_url}{movie_search_page.find('a')['href']}"
  search_box_info['movie_id'] = None if not search_box_info['movie_page_url'] else search_box_info['movie_page_url'].split('/')[-2]
  search_box_info['movie_name'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['alt'].strip()
  search_box_info['movie_thubmnail'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['src']
  search_box_info['movie_year'] = None if not movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}) else movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','')
  search_box_info['movie_rating'] = None if not movie_search_page.find('strong') else movie_search_page.find('strong').text
  search_box_info['movie_metascore'] = None if not movie_search_page.find('span', {'class': 'metascore'}) else movie_search_page.find('span', {'class': 'metascore'}).text.strip()
  search_box_info['movie_description'] = None if not movie_search_page.find_all('p',{'class':'text-muted'}) else movie_search_page.find_all('p',{'class':'text-muted'})[1].text.strip()
  search_box_info['runtime_min'] = None if not movie_search_page.find('span',{'class':'runtime'}) else movie_search_page.find('span',{'class':'runtime'}).text.split(' ')[0]

  bottom_box_info = movie_search_page.find('p',{'class':'sort-num_votes-visible'}).find_all('span')
  search_box_info['movie_vote_num'] = bottom_box_info[1]['data-value']
  search_box_info['movie_gross'] = bottom_box_info[4]['data-value'] if len(bottom_box_info)>2 else None;

  search_box_info['movie_rating'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}).text.strip()
  search_box_info['movie_genere'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}).text.strip()

  search_box_info['__SuccsefullyCollectBasicDetails'] = True;

  return search_box_info

In [7]:
movie_search_page = list_of_50_movies_soup[0]
search_box_info = get_basic_details(movie_search_page)
search_box_info



{'movie_place': '1',
 'movie_name': 'Everything Everywhere All at Once',
 'movie_page_url': 'https://www.imdb.com/title/tt6710474/?ref_=adv_li_i',
 'movie_id': 'tt6710474',
 'movie_thubmnail': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
 'movie_year': '2022',
 'movie_rating': '12',
 'movie_metascore': '81',
 'movie_description': 'A middle-aged Chinese immigrant is swept up into an insane adventure in which she alone can save existence by exploring other universes and connecting with the lives she could have led.',
 'runtime_min': '139',
 'movie_vote_num': '409145',
 'movie_gross': '72,861,480',
 'movie_genere': 'Action, Adventure, Comedy',
 '__SuccsefullyCollectBasicDetails': True}

In [8]:
movie_main_page = requests.get(search_box_info['movie_page_url'], headers=headers)
movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')

In [9]:
def get_extended_details(movie_main_page_soup):
  movie_page_dict = {}

  director_box = movie_main_page_soup.find('span',string="Director") if movie_main_page_soup.find('span',string="Directors") is None else movie_main_page_soup.find('span',string="Directors")
  if director_box == None:
    director_box = movie_main_page_soup.find('a',string="Director") if movie_main_page_soup.find('span',string="Directors") is None else movie_main_page_soup.find('a',string="Directors")
  director_box = director_box.find_next_sibling()
  movie_page_dict['director_name'] = director_box.find('a').text
  movie_page_dict['director_url'] = f"{base_url}{director_box.find('a')['href']}"
  movie_page_dict['director_id'] = movie_page_dict['director_url'].split('/')[-2]

  writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('span',string="Writers") is None else movie_main_page_soup.find('span',string="Writers")
  if writers_box == None:
    writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('a',string="Writers") is None else movie_main_page_soup.find('a',string="Writers")
  writers_box = writers_box.find_next_sibling()
  movie_page_dict['writer_name'] = director_box.find('a').text
  movie_page_dict['writer_url'] = f"{base_url}{writers_box.find('a')['href']}"
  movie_page_dict['writer_id'] = movie_page_dict['writer_url'].split('/')[-2]

  stars_box = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}).find_parent().find_all('a')
  for i,star_box in enumerate(stars_box[1:-1]):
    star_num = f'star_{i+1}'
    movie_page_dict[f'{star_num}_name'] = star_box.text.strip()
    movie_page_dict[f'{star_num}_url'] = f"{base_url}{star_box['href']}"
    movie_page_dict[f'{star_num}_id'] = star_box['href'].split('/')[-2]

  review_scores = movie_main_page_soup.find_all('span',{'class':'score'})
  movie_page_dict['user_reviews_count'] = review_scores[0].text
  movie_page_dict['critic_reviews_count'] = review_scores[1].text

  movie_page_dict['release_date'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}, string="Release date").find_parent().find('a',{'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text.split('(')[0].strip()
  movie_page_dict['release_date']



  movie_page_dict['budget'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Budget")
  if (movie_page_dict['budget']!=None):
    movie_page_dict['budget'] = movie_page_dict['budget'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['gross_worldwide'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross worldwide")
  if (movie_page_dict['gross_worldwide']!=None):
    movie_page_dict['gross_worldwide'] = movie_page_dict['gross_worldwide'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['gross_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross US & Canada")
  if (movie_page_dict['gross_us_canada']!=None):
    movie_page_dict['gross_us_canada'] = movie_page_dict['gross_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['opening_weekend_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Opening weekend US & Canada")
  if (movie_page_dict['opening_weekend_us_canada']!=None):
    movie_page_dict['opening_weekend_us_canada'] = movie_page_dict['opening_weekend_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

  movie_page_dict['origin_country'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Country of origin")
  if (movie_page_dict['origin_country']!=None):
    movie_page_dict['origin_country'] = movie_page_dict['origin_country'].find_next_sibling().find('a').text.split('(')[0].strip()

  languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
  languages = [] if not languages else languages.find_next_sibling().find_all('a')
  movie_page_dict['languages'] = ",".join([language.text for language in languages])

  movie_page_dict['__SuccsefullyCollectExtandedDetails'] = True;

  return movie_page_dict

get_extended_details(movie_main_page_soup)

{'director_name': 'Daniel Kwan',
 'director_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_dr',
 'director_id': 'nm3453283',
 'writer_name': 'Daniel Kwan',
 'writer_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_wr',
 'writer_id': 'nm3453283',
 'star_1_name': 'Michelle Yeoh',
 'star_1_url': 'https://www.imdb.com/name/nm0000706/?ref_=tt_ov_st',
 'star_1_id': 'nm0000706',
 'star_2_name': 'Stephanie Hsu',
 'star_2_url': 'https://www.imdb.com/name/nm3513533/?ref_=tt_ov_st',
 'star_2_id': 'nm3513533',
 'star_3_name': 'Jamie Lee Curtis',
 'star_3_url': 'https://www.imdb.com/name/nm0000130/?ref_=tt_ov_st',
 'star_3_id': 'nm0000130',
 'user_reviews_count': '3.3K',
 'critic_reviews_count': '394',
 'release_date': 'May 19, 2022',
 'budget': '$14,300,000',
 'gross_worldwide': '$135,559,660',
 'gross_us_canada': '$76,852,389',
 'opening_weekend_us_canada': '$501,305',
 'origin_country': 'United States',
 'languages': 'English,Mandarin,Cantonese',
 '__SuccsefullyCollectExtandedDetai

## Cursor for collecting movies in batches

In [10]:
# Fixed parameters:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [11]:
# Dynamic parameters:

start_point = 1 # Start from movie #1-50

movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&languages=en&start={start_point}&ref_=adv_nxt"
print(f"movie_search_url_50_batch: {movie_search_url_50_batch}")
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)

print(f"max_num_of_results: {max_num_of_results}")

movie_search_url_50_batch: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&1anguages=en&start=1&ref_=adv_nxt
max_num_of_results: 6286


In [12]:
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)
max_num_of_results

6286

In [22]:
total_movie_dataset = []
total_actor_dataset = []
total_director_dataset = []

start_point = 1 # Start from movie #1-50

while start_point < max_num_of_results-50:

  movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&languages=en&start={start_point}&ref_=adv_nxt"
  print(f"movie_search_url_50_batch start point {start_point}: {movie_search_url_50_batch}")
  search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
  search_page_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser')
  list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

  for i,movie in enumerate(list_of_50_movies_soup):
    movie_data = {}

    basic_data = get_basic_details(movie)
    movie_data.update(basic_data)

    movie_main_page = requests.get(basic_data['movie_page_url'], headers=headers)
    movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

    if i%30==0:
      print(basic_data['movie_name'],basic_data['movie_page_url'])

    extended_deta = get_extended_details(movie_main_page_soup)
    movie_data.update(extended_deta)

    sleep(0.1)

    total_movie_dataset.append(movie_data)

  movie_dataset = pd.DataFrame(total_movie_dataset)
  movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

  start_point += 50

movie_search_url_50_batch start point 1: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&languages=en&start=1&ref_=adv_nxt
Everything Everywhere All at Once https://www.imdb.com/title/tt6710474/?ref_=adv_li_i
Pinocchio https://www.imdb.com/title/tt1488589/?ref_=adv_li_i
movie_search_url_50_batch start point 51: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&languages=en&start=51&ref_=adv_nxt
Everything Everywhere All at Once https://www.imdb.com/title/tt6710474/?ref_=adv_li_i
Pinocchio https://www.imdb.com/title/tt1488589/?ref_=adv_li_i
movie_search_url_50_batch start point 101: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&languages=en&start=101&ref_=adv_nxt
Everything Everywhere All at Once https://www.imdb.com/title/tt6710474/?ref_=adv_li_i
Pinocchio https://www.imdb.com/title/tt1488589/?ref_=adv_li_i
movie_search_url

ConnectionError: ignored

In [18]:
movie_dataset = pd.DataFrame(total_movie_dataset)
movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

movie_dataset

,movie_place,movie_name,movie_page_url,movie_id,movie_thubmnail,movie_year,movie_rating,movie_metascore,movie_description,runtime_min,...,user_reviews_count,critic_reviews_count,release_date,budget,gross_worldwide,gross_us_canada,opening_weekend_us_canada,origin_country,languages,__SuccsefullyCollectExtandedDetails
0,1,Everything Everywhere All at Once,https://www.imdb.com/title/tt6710474/?ref_=adv...,tt6710474,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,81,A middle-aged Chinese immigrant is swept up in...,139,...,3.3K,394,"May 19, 2022","$14,300,000","$135,559,660","$76,852,389","$501,305",United States,"English,Mandarin,Cantonese",True
1,2,The Whale,https://www.imdb.com/title/tt13833688/?ref_=ad...,tt13833688,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,60,"A reclusive, morbidly obese English teacher at...",117,...,669,266,"February 16, 2023","$10,000,000","$53,063,485","$17,424,083","$332,152",United States,English,True
2,3,The Banshees of Inisherin,https://www.imdb.com/title/tt11813216/?ref_=ad...,tt11813216,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,16,87,Two lifelong friends find themselves at an imp...,114,...,961,326,"January 26, 2023",None,"$48,659,132","$10,582,266","$184,454",None,English,True
3,4,Triangle of Sadness,https://www.imdb.com/title/tt7322224/?ref_=adv...,tt7322224,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,63,A fashion model celebrity couple join an event...,147,...,437,259,"December 15, 2022","€10,000,000","$25,377,810","$4,608,096","$214,602",None,"English,Swedish,German,French,Greek,Tagalog,Akan",True
4,5,Babylon,https://www.imdb.com/title/tt10640346/?ref_=ad...,tt10640346,https://m.media-amazon.com/images/S/sash/4Fyxw...,I 2022,16,60,A tale of outsized ambition and outrageous exc...,189,...,703,275,"January 19, 2023","$78,000,000","$63,372,177","$15,351,455","$3,603,368",United States,"English,Spanish,Italian,Cantonese,French,Hunga...",True
5,6,A Man Called Otto,https://www.imdb.com/title/tt7405458/?ref_=adv...,tt7405458,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,51,Otto is a grump who's given up on life followi...,126,...,316,156,"February 9, 2023","$50,000,000","$109,028,819","$64,228,819","$56,257",None,"English,Spanish",True
6,7,Puss in Boots: The Last Wish,https://www.imdb.com/title/tt3915174/?ref_=adv...,tt3915174,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,6,73,When Puss in Boots discovers that his passion ...,102,...,558,153,"December 21, 2022","$90,000,000","$476,094,321","$184,448,235","$12,429,515",None,"English,Spanish",True
7,8,Im Westen nichts Neues,https://www.imdb.com/title/tt1016150/?ref_=adv...,tt1016150,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,16,76,A young German soldier's terrifying experience...,148,...,731,191,"October 28, 2022",None,None,None,None,None,"German,French",True
8,9,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/?ref_=adv...,tt1630029,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,67,Jake Sully lives with his newfound family form...,192,...,2.7K,353,"December 14, 2022","$350,000,000","$2,310,214,014","$680,952,437","$134,100,226",United States,English,True
9,10,The Unbearable Weight of Massive Talent,https://www.imdb.com/title/tt11291274/?ref_=ad...,tt11291274,https://m.media-amazon.com/images/S/sash/4Fyxw...,2022,12,68,Moviestar Nick Cage is channeling his iconic c...,107,...,678,275,"April 21, 2022","$30,000,000","$29,116,320","$20,300,157","$7,131,703",United States,"English,Spanish",True


In [15]:
languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
languages = [] if not languages else languages.find_next_sibling().find_all('a')
",".join([language.text for language in languages])

'English'

In [16]:
# movie_main_page_soup
# writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('a',string="Writers") is None else movie_main_page_soup.find('a',string="Writers")
# writers_box = writers_box.find_next_sibling()
# writers_box